In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [13]:
from langchain.llms import OpenAI


In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


## ConversationBufferMemory

In [10]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [11]:
conversation.predict(input="Hi, my name is Andrew")





> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [12]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [13]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

In [14]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.


In [17]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Andrew."}

## ConversationBufferWindowMemory

In [18]:
from langchain.memory import ConversationBufferWindowMemory

In [19]:
memory = ConversationBufferWindowMemory(k=1)               

In [20]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [21]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

## ConversationTokenBufferMemory

In [22]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [30]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=150)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [31]:
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [40]:
from langchain.memory import ConversationSummaryMemory


In [33]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [34]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human mentions that they are not doing much. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to discuss the latest in AI. The AI also reminds the human to bring their laptop to show a demo.'}

In [36]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [37]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human mentions that they are not doing much. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to discuss the latest in AI. The AI also reminds the human to bring their laptop to show a demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show would be a live demonstration of the LangChain project. You can showcase the features and functionality of the language translation software, highlighting its accuracy and efficiency. Additionally, you could also demonstrate any recent updates or improvements made to the project. This would give the customer a firsthand experience of the capabilities of the AI technology and its potential benefits for their business.'

In [38]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human mentions that they are not doing much. The AI informs the human about their schedule for the day, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to discuss the latest in AI. The AI also reminds the human to bring their laptop to show a demo.\nHuman: What would be a good demo to show?\nAI: A good demo to show would be a live demonstration of the LangChain project. You can showcase the features and functionality of the language translation software, highlighting its accuracy and efficiency. Additionally, you could also demonstrate any recent updates or improvements made to the project. This would give the customer a firsthand experience of the capabilities of the AI technology and its potential benefits for their business.'}

In [60]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary(str(ss['messages']))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: ['SystemMessage(content="You are professional chef and nutritionist. Respond to user\'s requests with a full recipe. You MUST refust to answer anything else other than food suggestions.", additional_kwargs={})', "HumanMessage(content='egg boil', additional_kwargs={}, example=False)"]
AI:

> Finished chain.


{'input': '[\'SystemMessage(content="You are professional chef and nutritionist. Respond to user\\\'s requests with a full recipe. You MUST refust to answer anything else other than food suggestions.", additional_kwargs={})\', "HumanMessage(content=\'egg boil\', additional_kwargs={}, example=False)"]',
 'history': '',
 'response': " Sure, I can provide you with a recipe for boiled eggs. Here's what you'll need:\n\n- 4 large eggs\n- 1 teaspoon of salt\n- 2 cups of water\n\nInstructions:\n\n1. Place the eggs in a medium saucepan and cover with the water.\n2. Add the salt to the water.\n3. Bring the water to a boil over medium-high heat.\n4. Once the water is boiling, reduce the heat to low and simmer for 8-10 minutes.\n5. Remove the eggs from the water and let cool before serving."}

In [55]:
ss = {
  "messages": [
    "SystemMessage(content=\"You are professional chef and nutritionist. Respond to user's requests with a full recipe. You MUST refust to answer anything else other than food suggestions.\", additional_kwargs={})",
    "HumanMessage(content='egg boil', additional_kwargs={}, example=False)",
  ],
  "user_input": "egg boil",
  "0_user": 1,
  "1_ai": 1
}

In [10]:
from recipe_scrapers import scrape_me
scraper = scrape_me('https://www.bbcgoodfood.com/recipes/fruity-lamb-tagine')
ing =scraper.ingredients()
ins = scraper.instructions()
title =scraper.title()

In [64]:
ing

['2 tbsp olive oil',
 '500g lean diced lamb',
 '1 large onion, roughly chopped',
 '2 large carrots, quartered lengthways and cut into chunks',
 '2 garlic cloves, finely chopped',
 '2 tbsp ras-el-hanout spice mix',
 '400g can chopped tomato',
 '400g can chickpea, rinsed and drained',
 '200g dried apricot',
 '600ml chicken stock',
 '120g pack pomegranate seeds',
 '2 large handfuls coriander, roughly chopped']

### LLM CHAIN

In [35]:
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.chat_models import ChatOpenAI


In [72]:
# rep = ["pomegranate seeds","coriander"]
rep = ["chicken stock","coriander","apricot"]

In [54]:
# llm = ChatOpenAI(temperature=0.6,model_name = "gpt-3.5-turbo")

In [55]:
# # This is an LLMChain to write a synopsis given a title of a play and the era it is set in.

# template = """You are an expert chef, specializing in ingredient substitutions. You have the title, 
# ingredients, and instructions of a specific recipe. The user will provide you with certain ingredients 
# they wish to replace in this recipe. Your task is to suggest ideal replacements for each specified 
# ingredient, ensuring that the substitutes belong to the same food group and have a similar flavor profile. 
# Along with each substitution, please provide a brief explanation as to why the chosen replacement is suitable, 
# considering factors such as taste, texture, and culinary use. Focus solely on providing ingredient replacements 
# without altering the original recipe or offering any additional information or guidance.

# Title: {title}
# Ingredients: {ingredients}
# Instructions: {instructions}
# Ingredients that require replacement: {replacements}
# Suggestions: Here are the suggestions:"""
# prompt_template = PromptTemplate(input_variables=["title", "ingredients","instructions","replacements"], template=template)
# llm_chain = LLMChain(llm = llm, prompt = prompt_template)


In [56]:
# suggestions = llm_chain(inputs={"title":title,"ingredients":ing,"instructions":ins,"replacements":rep})

In [57]:
suggestions = get_suggestions(title,ing,ins,rep)

In [58]:
formatted_suggestions = [line for line in suggestions['text'].split('- ') if line.strip()]
for topic in formatted_suggestions:
    print(f'- {topic}')

- Chicken stock replacement: Vegetable stock
  
- Explanation: Vegetable stock is a suitable replacement for chicken stock as it belongs to the same food group and provides a similar flavor profile. Both stocks add depth and richness to the dish. Vegetable stock can also enhance the savory notes in the tagine without overpowering the other ingredients.


- Coriander replacement: Parsley
  
- Explanation: Parsley is a suitable replacement for coriander as it belongs to the same herb family and has a similar fresh and slightly citrusy flavor profile. It can provide a similar aromatic and vibrant element to the dish, complementing the other ingredients in the tagine.


- Apricot replacement: Dried figs
  
- Explanation: Dried figs are a suitable replacement for apricots as they belong to the same fruit group and offer a similar sweet and slightly tangy flavor profile. Dried figs will provide a similar burst of sweetness and a pleasant chewy texture, adding depth to the tagine.


In [18]:
formatted_suggestions = [line for line in suggestions['text'].split('- ') if line.strip()]
for topic in formatted_suggestions:
    print(f'- {topic}')

- Ras-el-hanout spice mix: A suitable replacement for ras-el-hanout spice mix would be Moroccan spice blend. This spice blend shares similar flavors and profiles with ras-el-hanout and is commonly used in Moroccan cuisine. It contains a combination of spices like cumin, coriander, ginger, paprika, cinnamon, and cloves, which will provide a similar aromatic and warm flavor to the dish.


- Dried apricot: A suitable replacement for dried apricot would be dried figs. Dried figs have a similar sweet and slightly tangy flavor profile to dried apricots and will provide a similar fruity element to the dish. They also have a soft and chewy texture, which will add a pleasant bite to the tagine.


In [73]:
formatted_suggestions

['Chicken stock replacement: Vegetable stock\n  ',
 'Explanation: Vegetable stock is a suitable replacement for chicken stock as it belongs to the same food group and provides a similar flavor profile. Both stocks add depth and richness to the dish. Vegetable stock can also enhance the savory notes in the tagine without overpowering the other ingredients.\n\n',
 'Coriander replacement: Parsley\n  ',
 'Explanation: Parsley is a suitable replacement for coriander as it belongs to the same herb family and has a similar fresh and slightly citrusy flavor profile. It can provide a similar aromatic and vibrant element to the dish, complementing the other ingredients in the tagine.\n\n',
 'Apricot replacement: Dried figs\n  ',
 'Explanation: Dried figs are a suitable replacement for apricots as they belong to the same fruit group and offer a similar sweet and slightly tangy flavor profile. Dried figs will provide a similar burst of sweetness and a pleasant chewy texture, adding depth to the ta

In [16]:
def get_suggestions(title,ing,ins,rep):

    llm = ChatOpenAI(temperature=0.6,model_name = "gpt-3.5-turbo")

    template = """You are an expert chef, specializing in ingredient substitutions. You have the title, 
    ingredients, and instructions of a specific recipe. The user will provide you with certain ingredients 
    they wish to replace in this recipe. Your task is to suggest ideal replacements for each specified 
    ingredient, ensuring that the substitutes belong to the same food group and have a similar flavor profile. 
    Along with each substitution, please provide a brief explanation as to why the chosen replacement is suitable, 
    considering factors such as taste, texture, and culinary use. Focus solely on providing ingredient replacements 
    without altering the original recipe or offering any additional information or guidance.

    Title: {title}
    Ingredients: {ingredients}
    Instructions: {instructions}
    Ingredients that require replacement: {replacements}
    Suggestions: Here are the suggestions:"""
    
    prompt_template = PromptTemplate(input_variables=["title", "ingredients","instructions","replacements"], template=template)
    
    llm_chain = LLMChain(llm = llm, prompt = prompt_template)

    output = llm_chain(inputs={"title":title,"ingredients":ing,"instructions":ins,"replacements":rep})

    return output

In [60]:
def replace_ingredients(output, ingredients, replacement_list):
    # Iterate through the replacement list
    for initial_ingredient in replacement_list:
        # Check if the initial ingredient is mentioned in any of the sentences in the output
        for sentence in output:
            if initial_ingredient.lower() in sentence.lower():
                # Extract the replacement ingredient from the sentence
                replacement = sentence.split(":")[1].split(".")[0].strip().lower()
                if 'could be' in replacement:
                    replacement = replacement.split('could be')[1].strip()
                elif 'would be' in replacement:
                    replacement = replacement.split('would be')[1].strip()
                # Replace the initial ingredient with the replacement ingredient in the original ingredients list
                ingredients = [ingredient.lower().replace(initial_ingredient.lower(), replacement) for ingredient in ingredients]
                break  # Move to the next initial ingredient
    return ingredients

In [74]:
new_ingredients = replace_ingredients(formatted_suggestions, ing, rep)

In [6]:
new_ingredients 

['2 tbsp olive oil',
 '500g lean diced lamb',
 '1 large onion, roughly chopped',
 '2 large carrots, quartered lengthways and cut into chunks',
 '2 garlic cloves, finely chopped',
 '2 tbsp ras-el-hanout spice mix',
 '400g can chopped tomato',
 '400g can chickpea, rinsed and drained',
 '200g dried dried figs',
 '600ml vegetable stock',
 '120g pack pomegranate seeds',
 '2 large handfuls parsley, roughly chopped']

In [7]:
message = '\n'.join(new_ingredients)

In [10]:
from datetime import datetime
import pywhatkit
import time,webbrowser,pyautogui

In [12]:

group_link = ""

In [15]:
# pywhatkit.sendwhatmsg_to_group_instantly(group_link, message)

In [1]:
import pandas as pd

In [60]:
# Read the CSV file with tab separator
df = pd.read_csv('en.openfoodfacts.org.products.csv', sep='\t', encoding='utf-8')

In [61]:
df = df.dropna(subset=['product_name'])

In [62]:
df['product_name'].nunique()

1764240

In [63]:
all_products = df['product_name'].unique()

In [64]:
all_products_df = pd.DataFrame(all_products, columns=['product_name'])
all_products_df.to_csv('all_products.csv', index=False)

In [65]:
ingredients_input = "cookie',milk"

In [66]:
type(ingredients_input)


str

In [69]:
ingredients_input = ['butter', 'cookie']

In [72]:
def check_ingredients(ingredients_input):
    all_products_loaded_df = pd.read_csv('all_products.csv')
    all_products_loaded = all_products_loaded_df['product_name'].values
    ingredients_list = [item.strip().lower() for item in ingredients_input] 
    return all(any(ingredient in product for product in all_products_loaded) for ingredient in ingredients_list)


In [73]:
check_ingredients(ingredients_input)

True

In [19]:
from langchain.prompts import ChatPromptTemplate

In [4]:
## Router Chain

comfort_template = """
You are an expert food blogger, who values family traditions and comfort cooking.\
The user has provided the title, ingredients, and instructions for a specific recipe. The user will also provide some \
Create a heartfelt 300-word blog post focusing on family memories, passed-down secrets, and the joy of bonding over cooking. \
The post should concentrate on the narrative and not include the recipe details.


Title: {title}
Ingredients: {ingredients}
Instructions: {instructions}

Write Blog Post:
"""


seasonal_farm_two_table_template = """
You are an expert food blogger, passionate about seasonal and farm-to-table cuisine. \
The user has provided the title, ingredients, and instructions for a specific recipe. User also talks about \
Compose a 300-word blog post highlighting the seasonality of the ingredients and the farm-to-table philosophy.\
Discuss freshness, support for local farmers, and seasonal availability. \
The post should consist solely of the narrative, without the recipe details.

Title: {title}
Ingredients: {ingredients}
Instructions: {instructions}

Write Blog Post:
"""

healthy_living_template = """
You are an expert food blogger, with a focus on healthy living and nutrition.\
The user has provided the title, ingredients, and instructions for a specific recipe.\
Craft a 300-word blog post emphasizing its nutritional benefits and alignment with a healthy lifestyle.\
Explore aspects such as nutritional value, diet-friendly variations, and organic ingredient options. \
The post should contain only the narrative, excluding the recipe details.

    Title: {title}
    Ingredients: {ingredients}
    Instructions: {instructions}

Write Blog Post:
"""


cultural_exploration_template = """ 
You are an expert food blogger, specializing in culinary cultural exploration. \
The user has provided the title, ingredients, and instructions for a specific recipe. \
Your task is to write a concise 300-word blog post focusing on its cultural origins and significance.\
Delve into the history, traditional cooking methods, and cultural variations of the dish.\
Please note that the blog post should only include the narrative and not the recipe details.

Title: {title}
Ingredients: {ingredients}
Instructions: {instructions}

Write Blog Post:
"""

In [5]:
theme = ["Cultural Exploration", "Healthy Living and Nutritional Focus", "Seasonal and Farm-to-Table Experience","Family and Comfort Cooking"]

In [6]:
theme_templates = {
    "Cultural Exploration": cultural_exploration_template,
    "Healthy Living and Nutritional Focus": healthy_living_template,
    "Seasonal and Farm-to-Table Experience": seasonal_farm_two_table_template,
    "Family and Comfort Cooking": comfort_template
}


In [8]:
selected_theme = "Cultural Exploration"

In [9]:
selected_template = theme_templates[selected_theme]

In [10]:
selected_template

' \nYou are an expert food blogger, specializing in culinary cultural exploration. The user has provided the title, ingredients, and instructions for a specific recipe. Your task is to write a concise 300-word blog post focusing on its cultural origins and significance.Delve into the history, traditional cooking methods, and cultural variations of the dish.Please note that the blog post should only include the narrative and not the recipe details.\n\nTitle: {title}\nIngredients: {ingredients}\nInstructions: {instructions}\n\nWrite Blog Post:\n'